# Learning FLPO

In this code, we will implement the hierarchical ML architecture to predict the parameters in the FLPO setup.

In [2]:
import torch
from VRP_Net import VRPNet
from matplotlib import pyplot as plt
from utils import *
import os
import matplotlib.pyplot as plt
from torchinfo import summary

# Loading the Data

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Running on: " , device)
num_data = 10 # number of FLPO instances
num_nodes = 12
city_dim = 2
data = torch.rand(num_data,num_nodes,city_dim).to(device)

Running on:  cuda


# Loading the VRP NET Module

In [4]:
hidden_dim = 128
num_layers = 3
num_heads = 8
input_dim = 2
num_samples = 5 # for inference
vrpnet = VRPNet(input_dim,hidden_dim,device,num_heads=num_heads,num_enc_layers=num_layers,num_dec_layers=num_layers,use_PE=False)
vrpnet.load_state_dict(torch.load('Saved models/POMO2025_03_12 11_03_550.8357473611831665best_model.pth',weights_only=True))
vrpnet.eval()
print('VRP NET loaded.')
print(summary(vrpnet))

VRP NET loaded.
Layer (type:depth-idx)                                                 Param #
VRPNet                                                                 --
├─Encoder: 1-1                                                         --
│    └─Linear: 2-1                                                     384
│    └─PositionalEncoding: 2-2                                         --
│    └─TransformerEncoder: 2-3                                         --
│    │    └─ModuleList: 3-1                                            594,816
├─Decoder: 1-2                                                         --
│    └─TransformerDecoder: 2-4                                         --
│    │    └─ModuleList: 3-2                                            793,728
├─PositionalEncoding: 1-3                                              --
Total params: 1,388,928
Trainable params: 1,388,928
Non-trainable params: 0


# Generating d_min

In [6]:
all_samples = torch.zeros(num_data, num_nodes, num_samples).to(device)
for i in range(num_samples):
    _, actions = vrpnet(data, mod='train')
    all_samples[:,:,i:i+1] = actions
opt_sample_indices = []
d_mins = []
for data_idx in range(num_data):
    costs_for_data = [route_cost(data[data_idx:data_idx+1], all_samples[data_idx:data_idx+1, :, sample_idx]) for sample_idx in range(num_samples)]
    costs_tensor = torch.tensor(costs_for_data)
    d_min, opt_sample_index = torch.min(costs_tensor, dim=0)  
    opt_sample_indices.append(opt_sample_index.item())
    d_mins.append(d_min)  
opt_sample_indices = torch.tensor(opt_sample_indices)
actions = all_samples[torch.arange(num_data),:,opt_sample_indices]
print("d_mins for all data are: ",d_mins)

d_mins for all data are:  [tensor(0.0866), tensor(0.2088), tensor(0.7325), tensor(0.4038), tensor(0.5385), tensor(0.0077), tensor(0.6882), tensor(0.5132), tensor(0.9509), tensor(0.0915)]


# Loading Free Energy Network

Dhananjay's code for loading his DNN is here. The output of this block should be Y.

# Calculating Free Energy

Feeding the d_min from Salar Block to get the Free energy

# Calculating the gradients

The output of this block shoud be dF/dy

# Running CLF to determin Y

The GD code here

# Closing the Beta Loop for Annealing